# Funções em Python - ETL com Pandas, JSON e Parquet

Para realizar uma ETL (Extract, Transform, Load) simples utilizando Python e a biblioteca Pandas, vamos seguir os seguintes passos:

Extract: Ler os dados de um arquivo JSON.

Transform: Concatenar os dados extraídos em um único DataFrame e aplicar uma transformação. A transformação específica dependerá dos dados, mas vamos assumir uma operação simples como um exemplo.

Load: Salvar o DataFrame resultante em um arquivo CSV ou PARQUET.

## Selecionando a raiz do projeto:

In [3]:
import os 
os.getcwd()
os.chdir('/home/jcnok/bootcamps/bootcamp-jornada-de-dados_2024/aula_08')
os.getcwd()

'/home/jcnok/bootcamps/bootcamp-jornada-de-dados_2024/aula_08'

## Loguru: Simplificando o Registro em Python


O Loguru é uma biblioteca de registro (logging) para Python que simplifica bastante o processo de registro de mensagens de registro (logs) em aplicativos Python. Ele oferece uma API fácil de usar e poderosa para configurar e gerenciar logs em seus projetos.

### Principais Características

1. **API Simples**: O Loguru oferece uma API muito simples e intuitiva para registro de mensagens. Você pode facilmente configurar o registro com apenas algumas linhas de código.

2. **Níveis de Log Personalizáveis**: Ele fornece vários níveis de log predefinidos (por exemplo, DEBUG, INFO, WARNING, ERROR, etc.) e também permite a definição de níveis de log personalizados conforme necessário.

3. **Formatação Flexível**: Você pode personalizar facilmente o formato das mensagens de log de acordo com suas necessidades. O Loguru oferece suporte a formatação de mensagem flexível e extensível.

4. **Suporte a Threads e Processos**: Loguru lida automaticamente com problemas de concorrência em aplicativos que usam várias threads ou processos, garantindo que as mensagens de log sejam registradas corretamente e sem conflitos.

5. **Integração com Outras Ferramentas**: Ele se integra facilmente com outras ferramentas e bibliotecas, como Flask, Django, e outras ferramentas populares de registro em Python.

### Conclusão

O Loguru simplifica o processo de registro de mensagens de log em aplicativos Python, oferecendo uma API fácil de usar, flexibilidade na formatação de mensagens e uma série de recursos úteis para gerenciamento eficaz de logs. É uma escolha popular entre os desenvolvedores Python devido à sua simplicidade e poder.


### O que é Logging?

Logging é o processo de gravar mensagens que documentam os eventos que ocorrem durante a execução de um software. Essas mensagens podem indicar progresso da execução, falhas, erros, ou outras informações úteis. O logging é crucial para desenvolvimento e manutenção de software, pois permite aos desenvolvedores e administradores de sistema entender o que o aplicativo está fazendo, diagnosticar problemas e monitorar o desempenho em produção.


### Instalação:

In [4]:
!poetry add loguru -q

### Como usar:

* **Configuração Básica e Registro de Mensagens Simples:**

In [ ]:
from loguru import logger

# Configuração básica
logger.add("app.log", rotation="500 MB", level="INFO")

# Exemplos de registro de mensagens
logger.debug("Esta é uma mensagem de depuração")
logger.info("Esta é uma mensagem informativa")
logger.warning("Esta é uma mensagem de aviso")
logger.error("Esta é uma mensagem de erro")
logger.critical("Esta é uma mensagem crítica")


![img](../img/loguru_basic.png)

* **Personalizando o formato das mensagens:**

In [ ]:
from loguru import logger

# Configuração com formato personalizado
logger.add("app.log", format="{time} - {level} - {message}", level="INFO")

# Exemplo de registro de mensagem com contexto
logger.info("Usuário {user} fez login", user="Julio Okuda")

![img](../img/personalize_loguru.png)

* **Tratamento de exceções**

In [ ]:
from loguru import logger

try:
    # Algum código que pode gerar uma exceção
    resultado = 1 / 0
except Exception as e:
    # Registro da exceção com o método exception()
    logger.exception("Ocorreu um erro inesperado: {exception}", exception=str(e))


![img](../img/exception.png)

Usando logger.exception(), Loguru automaticamente captura e loga o traceback da exceção, o que é extremamente útil para diagnóstico de erros.

* **Vamos criar um decorador utilizando o Loguru para adicionar automaticamente logs a qualquer função Python. Isso nos permite registrar automaticamente quando uma função é chamada e quando ela termina, junto com qualquer informação relevante, como argumentos da função e o resultado retornado (ou exceção lançada).**

* **Agora, vamos ao código do decorador:**

In [15]:
from loguru import logger

def log_decorator(func):
    """
    Decorador para registro de chamadas de função com o Loguru.

    Args:
        func (callable): Função a ser decorada.

    Returns:
        callable: Função decorada.
    """
    def wrapper(*args, **kwargs):
        """
        Função interna que envolve a função original e realiza o registro.

        Args:
            *args: Argumentos posicionais passados para a função.
            **kwargs: Argumentos de palavras-chave passados para a função.

        Returns:
            Qualquer: Resultado da função original.

        Raises:
            Exception: Se a função original lançar uma exceção.
        """
        # Registra a chamada da função com os argumentos e palavras-chave
        logger.info(f"Chamando '{func.__name__}' com {args} e {kwargs}")
        
        try:
            # Chama a função original e captura o resultado
            result = func(*args, **kwargs)
            # Registra o retorno da função
            logger.info(f"'{func.__name__}' retornou {result}")
            return result
        except Exception as e:
            # Registra a exceção se a função original lançar uma exceção
            logger.exception(f"'{func.__name__}' lançou uma exceção: {e}")
            # Propaga a exceção para cima na cadeia de chamadas
            raise
    
    return wrapper


### Uso e Execução do Decorador de Registro com Loguru

O decorador `log_decorator` é uma ferramenta útil para registrar chamadas de função em aplicativos Python usando a biblioteca Loguru. Este decorador pode ser aplicado a qualquer função para automatizar o registro de suas chamadas e resultados.

#### Como Usar o Decorador

Para usar o decorador `log_decorator`, siga estas etapas:

1. **Importe o Decorador**: Importe o decorador `log_decorator` de onde ele estiver definido no seu código.

2. **Aplique o Decorador**: Aplique o decorador `log_decorator` à função que deseja registrar. Por exemplo:

    ```python
    @log_decorator
    def minha_funcao(parametro):
        # Corpo da função
        pass
    ```

    Isso irá decorar a função `minha_funcao` com o decorador de registro, permitindo que todas as suas chamadas sejam registradas automaticamente.

#### Execução do Decorador

Quando uma função decorada com `log_decorator` é chamada, o decorador entra em ação:

1. **Registro de Chamada**: O decorador registra a chamada da função, incluindo os argumentos passados.

2. **Execução da Função Original**: O decorador chama a função original com os argumentos fornecidos.

3. **Registro de Resultado**: Se a função for executada com sucesso, o decorador registra o resultado retornado.

4. **Gestão de Exceções**: Se a função lançar uma exceção durante a execução, o decorador registra a exceção e a propaga para cima na cadeia de chamadas.


### Como Utilizar o Decorador

Agora, veja como aplicar o `log_decorator` a uma função:

In [ ]:
@log_decorator
def soma(a, b):
    return a + b

@log_decorator
def falha():
    raise ValueError("Um erro intencional")

# Testando as funções decoradas
soma(5, 3)  # Isso irá logar a chamada e o retorno
try:
    falha()  # Isso irá logar a chamada e a exceção
except ValueError:
    pass  # Ignora a exceção para fins de demonstração

Ao decorar as funções `soma` e `falha` com `@log_decorator`, automaticamente logamos a entrada e saída (ou exceção) dessas funções sem alterar o corpo delas. Isso é especialmente útil para debugar, monitorar a performance de aplicações ou simplesmente manter um registro de quais funções estão sendo chamadas e com quais argumentos.



#### Benefícios do Uso de Decoradores com Loguru


O uso de decoradores em conjunto com o Loguru fornece uma abordagem elegante e poderosa para adicionar logs a aplicações Python. Sem a necessidade de modificar o corpo da função, podemos facilmente adicionar funcionalidades de logging, o que torna o código mais limpo, mantém a separação de preocupações e facilita a manutenção e o debugging.

Além disso, ao centralizar a lógica de logging no decorador, promovemos a reutilização de código e garantimos uma forma consistente de logar informações através de diferentes partes de uma aplicação.


### Conclusão


O Loguru oferece uma abordagem moderna e conveniente para logging em Python, simplificando muitos aspectos que requerem configuração manual detalhada com o módulo de logging padrão do Python. Seja para desenvolvimento, depuração ou produção, adicionar logging ao seu aplicativo com Loguru pode melhorar significativamente a visibilidade e a capacidade de diagnóstico do seu código.


### Desafio

![img](../img/pic_05.png)

In [27]:
import os
import pandas as pd 
import glob 

def ler_arquivos_json(path_origin):
    caminho_arquivos = os.path.join(path_origin, '*.json')
    arquivos_json = glob.glob(caminho_arquivos)
    if not arquivos_json:
        raise ValueError("Nenhum arquivo JSON encontrado na pasta especificada.")
    dfs = [pd.read_json(arquivo) for arquivo in arquivos_json]
    return pd.concat(dfs, ignore_index=False)

# Função de transformação: adiciona uma nova coluna 'Receita' ao DataFrame
def transformar_dataframe(df):
    df['Receita'] = df['Quantidade'] * df['Venda']
    return df

# Função de carregamento: salva o DataFrame em CSV ou Parquet, dependendo do formato especificado
def carregar_dataframe(df, path_to_save, format_to_save):         
    
    for formato in format_to_save:
        if formato.lower() == 'csv':
            caminho_salvar_csv = path_to_save + '.csv'
            df.to_csv(caminho_salvar_csv, index=False)
            print(f"DataFrame salvo em '{caminho_salvar_csv}'")
        elif formato.lower() == 'parquet':
            caminho_salvar_parquet = path_to_save + '.parquet'
            df.to_parquet(caminho_salvar_parquet, index=False)
            print(f"DataFrame salvo em '{caminho_salvar_parquet}'")
        else:
            raise ValueError("Formato especificado não suportado. Use 'csv' ou 'parquet'.")


# Função principal
def pipeline(path_origin, path_to_save, format_to_save):
    # Etapa de extração
    df = ler_arquivos_json(path_origin)
    
    # Etapa de transformação
    df = transformar_dataframe(df)
    
    # Etapa de carregamento
    carregar_dataframe(df, path_to_save, format_to_save)

if __name__ == "__main__":
    # Define a pasta de origem dos arquivos JSON e o formato de destino ('csv' ou 'parquet')
    path_to_save = 'data/process/dados_processados'
    path_origin = 'data/'
    format_to_save = ['csv']  # ou 'parquet'
    
    # Executa o processo de ETL
    pipeline(path_origin, path_to_save, format_to_save)


DataFrame salvo em 'data/process/dados_processados.csv'


In [22]:
import pandas as pd 
import glob 
def ler_arquivos_json(pasta):
    caminho_arquivos = os.path.join(pasta, '*.json')
    arquivos_json = glob.glob(caminho_arquivos)
    if not arquivos_json:
        raise ValueError("Nenhum arquivo JSON encontrado na pasta especificada.")
    dfs = [pd.read_json(arquivo) for arquivo in arquivos_json]
    return pd.concat(dfs, ignore_index=False)

In [23]:
ler_arquivos_json('data/')

,Produto,Categoria,Quantidade,Venda,Data
0,Notebook Gamer,Eletrônicos,3,1500,2023-01-15
1,Mouse Sem Fio,Eletrônicos,10,30,2023-01-15
2,Teclado Mecânico,Eletrônicos,5,100,2023-01-15
0,Notebook Gamer,Eletrônicos,2,1500,2023-01-17
1,Mouse Sem Fio,Eletrônicos,14,30,2023-01-17
2,Teclado Mecânico,Eletrônicos,2,100,2023-01-17
0,Notebook Gamer,Eletrônicos,7,1500,2023-01-16
1,Mouse Sem Fio,Eletrônicos,10,30,2023-01-16
2,Teclado Mecânico,Eletrônicos,3,100,2023-01-16
